In [ ]:
#Section C Data Preprocessing...........

# Convert 'flightDate' to datetime
flightpredict_df['flightDate'] = pd.to_datetime(flightpredict_df['flightDate'])

# Convert 'travelDuration' from ISO 8601 duration format to total minutes
flightpredict_df['travelDuration'] = flightpredict_df['travelDuration'].apply(lambda x: pd.Timedelta(x).total_seconds() / 60)

# Split 'segmentsAirlineName' to create a new column 'AirlineName' keeping only the first part of the segment
flightpredict_df['airlineName'] = flightpredict_df['segmentsAirlineName'].str.split('|', expand=True, n=1)[0]

#Extracting Number of stops
flightpredict_df['numberOfStops'] = flightpredict_df['segmentsAirlineName'].apply(lambda segments: len(segments.split('||')) - 1)

#Converting Boolean to Int
for column in ['isBasicEconomy', 'isRefundable', 'isNonStop']:
    flightpredict_df[column] = flightpredict_df[column].astype(int)

#Formatting the Flight Date and extracting into multiple columns
flightpredict_df['flightMonth'] = flightpredict_df['flightDate'].dt.month
flightpredict_df['flightDay'] = flightpredict_df['flightDate'].dt.day
flightpredict_df['flightYear'] = flightpredict_df['flightDate'].dt.year

flightpredict_df.head()

,flightDate,segmentsAirlineName,segmentsArrivalTimeEpochSeconds,segmentsDepartureTimeEpochSeconds,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,airlineName,numberOfStops,flightMonth,flightDay,flightYear
0,2022-04-21,Delta||Delta,1650551340||1650563100,1650542400||1650554580,LGA,DFW,345.0,1,0,0,110.70,142.6,9,1487.0,Delta,1,4,21,2022
1,2022-04-24,American Airlines,1650853920,1650845400,ATL,LGA,142.0,0,0,1,450.23,498.6,1,762.0,American Airlines,0,4,24,2022
2,2022-04-22,Alaska Airlines||Alaska Airlines,1650678600||1650697140,1650671280||1650688200,OAK,LAX,431.0,0,0,0,645.02,717.0,7,1628.0,Alaska Airlines,1,4,22,2022
3,2022-05-18,Delta||Delta,1652914020||1652941920,1652904600||1652923320,LGA,SFO,622.0,0,0,0,440.93,497.6,9,2897.0,Delta,1,5,18,2022
4,2022-05-06,JetBlue Airways||JetBlue Airways,1651919400||1651929660,1651899120||1651925100,SFO,BOS,509.0,0,0,0,506.05,567.6,3,2751.0,JetBlue Airways,1,5,6,2022


In [ ]:
#Creating Distionary to extract the Airport Name and City Name from another dataframe

us_airports_df = airports_df[airports_df['COUNTRY'] == 'USA']
# Create the dictionary
airport_codes = us_airports_df.set_index('IATA')[['AIRPORT', 'CITY']].to_dict(orient='index')
# Standardize keys for clarity
airport_codes = {
    code: {'airport_name': details['AIRPORT'], 'city_name': details['CITY']}
    for code, details in airport_codes.items()
}

# Mapping Starting Airport Name and City from Dictionary
flightpredict_df['startingAirportName'] = flightpredict_df['startingAirport'].map(lambda x: airport_codes.get(x, {}).get('airport_name', 'Unknown'))
flightpredict_df['startingCityName'] = flightpredict_df['startingAirport'].map(lambda x: airport_codes.get(x, {}).get('city_name', 'Unknown'))

# Mapping Destination Airport Name and City from Dictionary
flightpredict_df['destinationAirportName'] = flightpredict_df['destinationAirport'].map(lambda x: airport_codes.get(x, {}).get('airport_name', 'Unknown'))
flightpredict_df['destinationCityName'] = flightpredict_df['destinationAirport'].map(lambda x: airport_codes.get(x, {}).get('city_name', 'Unknown'))

flightpredict_df.head()

,flightDate,segmentsAirlineName,segmentsArrivalTimeEpochSeconds,segmentsDepartureTimeEpochSeconds,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,...,totalTravelDistance,airlineName,numberOfStops,flightMonth,flightDay,flightYear,startingAirportName,startingCityName,destinationAirportName,destinationCityName
0,2022-04-21,Delta||Delta,1650551340||1650563100,1650542400||1650554580,LGA,DFW,345.0,1,0,0,...,1487.0,Delta,1,4,21,2022,LaGuardia,New York,Dallas-Fort Worth International,Dallas-Fort Worth
1,2022-04-24,American Airlines,1650853920,1650845400,ATL,LGA,142.0,0,0,1,...,762.0,American Airlines,0,4,24,2022,William B Hartsfield-Atlanta Intl,Atlanta,LaGuardia,New York
2,2022-04-22,Alaska Airlines||Alaska Airlines,1650678600||1650697140,1650671280||1650688200,OAK,LAX,431.0,0,0,0,...,1628.0,Alaska Airlines,1,4,22,2022,Metropolitan Oakland International,Oakland,Los Angeles International,Los Angeles
3,2022-05-18,Delta||Delta,1652914020||1652941920,1652904600||1652923320,LGA,SFO,622.0,0,0,0,...,2897.0,Delta,1,5,18,2022,LaGuardia,New York,San Francisco International,San Francisco
4,2022-05-06,JetBlue Airways||JetBlue Airways,1651919400||1651929660,1651899120||1651925100,SFO,BOS,509.0,0,0,0,...,2751.0,JetBlue Airways,1,5,6,2022,San Francisco International,San Francisco,Gen Edw L Logan Intl,Boston


In [ ]:
# Function to get the last segment from segmentsArrivalTimeEpochSeconds
def get_last_segment(arrival_time):
    return arrival_time.split("||")[-1]

# Function to get the first segment from segmentsDepartureTimeEpochSeconds
def get_first_segment(departure_time):
    return departure_time.split("||")[0]

# Create new columns with the last and first segments
flightpredict_df['lastSegmentArrival'] = flightpredict_df['segmentsArrivalTimeEpochSeconds'].apply(get_last_segment)
flightpredict_df['firstSegmentDeparture'] = flightpredict_df['segmentsDepartureTimeEpochSeconds'].apply(get_first_segment)

flightpredict_df.head()

,flightDate,segmentsAirlineName,segmentsArrivalTimeEpochSeconds,segmentsDepartureTimeEpochSeconds,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,...,numberOfStops,flightMonth,flightDay,flightYear,startingAirportName,startingCityName,destinationAirportName,destinationCityName,lastSegmentArrival,firstSegmentDeparture
0,2022-04-21,Delta||Delta,1650551340||1650563100,1650542400||1650554580,LGA,DFW,345.0,1,0,0,...,1,4,21,2022,LaGuardia,New York,Dallas-Fort Worth International,Dallas-Fort Worth,1650563100,1650542400
1,2022-04-24,American Airlines,1650853920,1650845400,ATL,LGA,142.0,0,0,1,...,0,4,24,2022,William B Hartsfield-Atlanta Intl,Atlanta,LaGuardia,New York,1650853920,1650845400
2,2022-04-22,Alaska Airlines||Alaska Airlines,1650678600||1650697140,1650671280||1650688200,OAK,LAX,431.0,0,0,0,...,1,4,22,2022,Metropolitan Oakland International,Oakland,Los Angeles International,Los Angeles,1650697140,1650671280
3,2022-05-18,Delta||Delta,1652914020||1652941920,1652904600||1652923320,LGA,SFO,622.0,0,0,0,...,1,5,18,2022,LaGuardia,New York,San Francisco International,San Francisco,1652941920,1652904600
4,2022-05-06,JetBlue Airways||JetBlue Airways,1651919400||1651929660,1651899120||1651925100,SFO,BOS,509.0,0,0,0,...,1,5,6,2022,San Francisco International,San Francisco,Gen Edw L Logan Intl,Boston,1651929660,1651899120


In [ ]:
# Function to convert epoch time to datetime and extract date, month, year, hour, minute, second
def extract_datetime_components_vectorized(epoch_series):
    dt_series = pd.to_datetime(epoch_series, unit='s', utc=True)
    return pd.DataFrame({
        'Date': dt_series.dt.day,
        'Month': dt_series.dt.month,
        'Year': dt_series.dt.year,
        'Hour': dt_series.dt.hour,
        'Minute': dt_series.dt.minute,
        'Second': dt_series.dt.second
    })

# Apply the vectorized function to the 'lastSegmentArrival' column
arrival_components = extract_datetime_components_vectorized(flightpredict_df['lastSegmentArrival'].astype(int))
arrival_components.columns = ['arrival' + col for col in arrival_components.columns]

# Apply the vectorized function to the 'firstSegmentDeparture' column
departure_components = extract_datetime_components_vectorized(flightpredict_df['firstSegmentDeparture'].astype(int))
departure_components.columns = ['departure' + col for col in departure_components.columns]

# Concatenate the new columns to the original DataFrame
flightpredict_df = pd.concat([flightpredict_df, arrival_components, departure_components], axis=1)

flightpredict_df.head()

,flightDate,segmentsAirlineName,segmentsArrivalTimeEpochSeconds,segmentsDepartureTimeEpochSeconds,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,...,arrivalYear,arrivalHour,arrivalMinute,arrivalSecond,departureDate,departureMonth,departureYear,departureHour,departureMinute,departureSecond
0,2022-04-21,Delta||Delta,1650551340||1650563100,1650542400||1650554580,LGA,DFW,345.0,1,0,0,...,2022,17,45,0,21,4,2022,12,0,0
1,2022-04-24,American Airlines,1650853920,1650845400,ATL,LGA,142.0,0,0,1,...,2022,2,32,0,25,4,2022,0,10,0
2,2022-04-22,Alaska Airlines||Alaska Airlines,1650678600||1650697140,1650671280||1650688200,OAK,LAX,431.0,0,0,0,...,2022,6,59,0,22,4,2022,23,48,0
3,2022-05-18,Delta||Delta,1652914020||1652941920,1652904600||1652923320,LGA,SFO,622.0,0,0,0,...,2022,6,32,0,18,5,2022,20,10,0
4,2022-05-06,JetBlue Airways||JetBlue Airways,1651919400||1651929660,1651899120||1651925100,SFO,BOS,509.0,0,0,0,...,2022,13,21,0,7,5,2022,4,52,0


In [ ]:
# Removing additional columns

flighttest_df = flightpredict_df.drop(['startingAirport', 'destinationAirport', 'firstSegmentDeparture', 'lastSegmentArrival', 'flightMonth', 'flightYear', 'flightDay', 'departureYear','arrivalYear', 'arrivalMonth', 'arrivalSecond', 'departureSecond','segmentsAirlineName', 'segmentsArrivalTimeEpochSeconds', 'segmentsDepartureTimeEpochSeconds'], axis=1)

# Function to classify the time
def classify_time(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

# Apply the function to create the new columns
flighttest_df['arrives'] = flighttest_df['arrivalHour'].apply(classify_time)
flighttest_df['departs'] = flighttest_df['departureHour'].apply(classify_time)

flighttest_df.head()

,flightDate,travelDuration,isBasicEconomy,isRefundable,isNonStop,baseFare,totalFare,seatsRemaining,totalTravelDistance,airlineName,...,destinationCityName,arrivalDate,arrivalHour,arrivalMinute,departureDate,departureMonth,departureHour,departureMinute,arrives,departs
0,2022-04-21,345.0,1,0,0,110.70,142.6,9,1487.0,Delta,...,Dallas-Fort Worth,21,17,45,21,4,12,0,Evening,Afternoon
1,2022-04-24,142.0,0,0,1,450.23,498.6,1,762.0,American Airlines,...,New York,25,2,32,25,4,0,10,Night,Night
2,2022-04-22,431.0,0,0,0,645.02,717.0,7,1628.0,Alaska Airlines,...,Los Angeles,23,6,59,22,4,23,48,Morning,Night
3,2022-05-18,622.0,0,0,0,440.93,497.6,9,2897.0,Delta,...,San Francisco,19,6,32,18,5,20,10,Morning,Evening
4,2022-05-06,509.0,0,0,0,506.05,567.6,3,2751.0,JetBlue Airways,...,Boston,7,13,21,7,5,4,52,Afternoon,Night
